In [ ]:
%load_ext autoreload
%autoreload 2
from common import *

In [ ]:
import json

RESULT_JSON = "/Users/law/repos/viper/results/all_ops/all_ops_full_revision.json"

results_raw = json.loads(open(RESULT_JSON).read())
# assert results_raw["context"]["library_build_type"] != "debug"
BENCHMARKS =  results_raw["benchmarks"]
# print(BENCHMARKS)

In [ ]:
from collections import defaultdict
runs = defaultdict(list)


BM_FIXTURES = [VIPER, DASH, PMEMKV, HYBRID_FASTER, UTREE, CRL_STORE, ROCKS]
BM_TYPES = ['insert', 'get', 'update', 'delete']

def get_bm_type(bm_type_str):
    for t in BM_TYPES:
        if t in bm_type_str:
            return t
    raise RuntimeError(f"Unknown bm_type: {bm_type_str}")

for bm in BENCHMARKS:
    found = False
    for (fixture, _) in BM_FIXTURES:
        bm_type = get_bm_type(bm['name'])
        if fixture in bm['name']:
            runs[(fixture, bm_type)].append(bm)
            found = True
            break
#     if not found: raise RuntimeError(f"Unknown fixture {bm['name']}")

for fixture, _ in BM_FIXTURES:
    for bm_type in BM_TYPES:
        runs[(fixture, bm_type)].sort(key=lambda x: x['threads'])

# pprint(runs)

In [ ]:
fig, (insert_ax, get_ax, update_ax, delete_ax) = plt.subplots(1, 4, figsize=(20, 3))


rcParams.update(plt.rcParamsDefault)
plt.style.use('default')

def get_bm_ax(bm_type):
    if bm_type == 'insert': return insert_ax
    if bm_type == 'get': return get_ax
    if bm_type == 'update': return update_ax
    if bm_type == 'delete': return delete_ax
    raise RuntimeError(f"Unknown bm_type {bm_type}")

num_threads = sorted({run['threads'] for run in list(runs.values())[0]})

for bm_type, letter in zip(BM_TYPES, ['a', 'b', 'c', 'd']):
    ax = get_bm_ax(bm_type)
    ax.set_xlabel("# Threads", fontsize=18)
    ax.set_title(f"({letter}) {bm_type.upper()}", fontsize=20)
    for (fixture, _) in BM_FIXTURES:
        r = [run['items_per_second'] / MILLION for run in runs[(fixture, bm_type)]]
        style = STYLES[fixture]
        x_vals = [1, 2, 4, 8, 16, 18] if fixture == CRL_STORE[0] and bm_type != 'get' else num_threads
        ax.plot(x_vals, r, marker=style.marker, color=style.color, lw=3, ms=8) 
                #, markeredgewidth=0.5, ms=style.marker_size,)
    x_ticks = np.append(ax.get_xticks(), 36)
    ax.set_xticks([1, 4, 8, 16, 24, 32, 36])
    ax.set_xlim(0, 37)
    ax.grid(axis='y', which='major')
    for tick in ax.xaxis.get_major_ticks():
        tick.label.set_fontsize(18)
    for tick in ax.yaxis.get_major_ticks():
        tick.label.set_fontsize(18)


insert_ax.set_ylabel("Throughput\n(Mops/s)", fontsize=18)
insert_ax.set_title(f"(a) PUT", fontsize=20)
insert_ax.set_ylim(0, 15)
get_ax.set_ylim(0, 35)
update_ax.set_ylim(0, 15)
delete_ax.set_ylim(0, 12)

insert_ax.set_yticks(range(0, 16, 3))
get_ax.set_yticks(range(0, 36, 5))
update_ax.set_yticks(range(0, 16, 3))
delete_ax.set_yticks(range(0, 13, 3))

fig.legend(loc='upper center', labels=[f[1] for f in BM_FIXTURES], 
           bbox_to_anchor=(0.5, 1.15), ncol=7, frameon=False, fontsize=18,
           columnspacing=1.3, handletextpad=0.3)
plt.tight_layout()

for ax in (insert_ax, get_ax, update_ax, delete_ax):
    hide_border(ax, True)

fig.savefig('charts/all_ops.pdf', bbox_inches='tight')
fig.savefig('charts/all_ops.svg', bbox_inches='tight')